# Description

It projects the PhenomeXcan results (S-MultiXcan, EFO version) into the MultiPLIER latent space.
Before projecting, repeated gene symbols as well as genes with NaN are removed;
additionally (also before projecting), S-MultiXcan results are adjusted for highly polygenic traits.

# Modules loading

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

from IPython.display import display
import pandas as pd

import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

import conf
from entity import Gene
from data.cache import read_data
from multiplier import MultiplierProjection

In [3]:
readRDS = ro.r["readRDS"]

In [4]:
saveRDS = ro.r["saveRDS"]

# Settings

In [5]:
RESULTS_PROJ_OUTPUT_DIR = Path(conf.RESULTS["PROJECTIONS_DIR"])
RESULTS_PROJ_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

display(RESULTS_PROJ_OUTPUT_DIR)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/projections')

# Load PhenomeXcan data (S-MultiXcan)

In [6]:
smultixcan_results_filename = conf.PHENOMEXCAN[
    "SMULTIXCAN_EFO_PARTIAL_MASHR_ZSCORES_FILE"
]

display(smultixcan_results_filename)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/data/phenomexcan/gene_assoc/smultixcan-efo_partial-mashr-zscores.pkl')

In [7]:
results_filename_stem = smultixcan_results_filename.stem
display(results_filename_stem)

'smultixcan-efo_partial-mashr-zscores'

In [8]:
smultixcan_results = pd.read_pickle(smultixcan_results_filename)

In [9]:
smultixcan_results.shape

(22515, 3752)

In [10]:
smultixcan_results.head()

,100001_raw-Food_weight,100002_raw-Energy,100003_raw-Protein,100004_raw-Fat,100005_raw-Carbohydrate,100006_raw-Saturated_fat,100007_raw-Polyunsaturated_fat,100008_raw-Total_sugars,100009_raw-Englyst_dietary_fibre,100010-Portion_size,...,visual impairment,vitiligo,vitreous body disease,vocal cord polyp,voice disorders,wellbeing measurement AND family relationship,wheezing,whooping cough,worry measurement,wrist fracture
gene_name,,,,,,,,,,,,,,,,,,,,,
ENSG00000000419,1.145442,0.724557,0.090876,0.298165,1.134347,1.371138,0.065718,0.794317,0.600342,0.317652,...,0.360518,1.351624,1.157695,0.835289,1.173072,1.337280,1.743822,1.017226,1.512184,0.972241
ENSG00000000457,0.618066,1.028131,2.218420,0.762584,0.934418,0.192993,1.080230,0.765997,0.375898,0.678731,...,2.134504,0.127830,0.534690,0.120516,0.517464,2.545363,0.673331,2.003092,0.344000,2.033122
ENSG00000000460,0.515724,0.403596,1.251359,0.433091,0.413466,0.246261,1.236151,0.827430,0.571985,0.782174,...,1.768905,0.992408,0.548215,0.412341,1.499415,1.366780,0.443318,0.417630,0.225934,1.613246
ENSG00000000938,0.280781,0.253910,0.879148,0.352705,0.051846,0.184212,0.148566,0.009989,0.363751,0.374514,...,0.656552,2.046041,2.746832,0.108211,1.008258,0.755695,0.896228,0.875047,0.476405,1.693057
ENSG00000000971,0.548127,0.389877,0.723469,1.167250,0.315952,0.324939,1.613932,0.311432,0.333548,1.807243,...,0.260482,0.646204,1.080240,0.678330,1.465358,0.307672,0.118376,1.419812,0.000002,1.040737


## Gene IDs to Gene names

In [11]:
smultixcan_results = smultixcan_results.rename(index=Gene.GENE_ID_TO_NAME_MAP)

In [12]:
smultixcan_results.shape

(22515, 3752)

In [13]:
smultixcan_results.head()

,100001_raw-Food_weight,100002_raw-Energy,100003_raw-Protein,100004_raw-Fat,100005_raw-Carbohydrate,100006_raw-Saturated_fat,100007_raw-Polyunsaturated_fat,100008_raw-Total_sugars,100009_raw-Englyst_dietary_fibre,100010-Portion_size,...,visual impairment,vitiligo,vitreous body disease,vocal cord polyp,voice disorders,wellbeing measurement AND family relationship,wheezing,whooping cough,worry measurement,wrist fracture
gene_name,,,,,,,,,,,,,,,,,,,,,
DPM1,1.145442,0.724557,0.090876,0.298165,1.134347,1.371138,0.065718,0.794317,0.600342,0.317652,...,0.360518,1.351624,1.157695,0.835289,1.173072,1.337280,1.743822,1.017226,1.512184,0.972241
SCYL3,0.618066,1.028131,2.218420,0.762584,0.934418,0.192993,1.080230,0.765997,0.375898,0.678731,...,2.134504,0.127830,0.534690,0.120516,0.517464,2.545363,0.673331,2.003092,0.344000,2.033122
C1orf112,0.515724,0.403596,1.251359,0.433091,0.413466,0.246261,1.236151,0.827430,0.571985,0.782174,...,1.768905,0.992408,0.548215,0.412341,1.499415,1.366780,0.443318,0.417630,0.225934,1.613246
FGR,0.280781,0.253910,0.879148,0.352705,0.051846,0.184212,0.148566,0.009989,0.363751,0.374514,...,0.656552,2.046041,2.746832,0.108211,1.008258,0.755695,0.896228,0.875047,0.476405,1.693057
CFH,0.548127,0.389877,0.723469,1.167250,0.315952,0.324939,1.613932,0.311432,0.333548,1.807243,...,0.260482,0.646204,1.080240,0.678330,1.465358,0.307672,0.118376,1.419812,0.000002,1.040737


## Remove duplicated gene entries

In [14]:
smultixcan_results.index[smultixcan_results.index.duplicated(keep="first")]

Index(['SPATA13', 'LINC01422', 'LINC00484', 'MAL2', 'GOLGA8M', 'LINC01115',
       'LYNX1'],
      dtype='object', name='gene_name')

In [15]:
smultixcan_results = smultixcan_results.loc[
    ~smultixcan_results.index.duplicated(keep="first")
]

In [16]:
smultixcan_results.shape

(22508, 3752)

## Some checks

In [17]:
# the data should have no NaN values
assert smultixcan_results.shape == smultixcan_results.dropna(how="any").shape

# Standardize S-MultiXcan results

Here we adjust for highly polygenic traits (see notebook `005_00-data_analysis.ipynb`): we penalize those traits that have large effect sizes across several genes, such as antropometric traits.

In [18]:
_tmp = smultixcan_results.apply(lambda x: x / x.sum())

In [19]:
_tmp.shape

(22508, 3752)

In [20]:
assert _tmp.shape == smultixcan_results.shape

In [21]:
# some testing
_trait = "body height"
_gene = "SCYL3"
assert (
    _tmp.loc[_gene, _trait]
    == smultixcan_results.loc[_gene, _trait] / smultixcan_results[_trait].sum()
)

_trait = "100001_raw-Food_weight"
_gene = "DPM1"
assert (
    _tmp.loc[_gene, _trait]
    == smultixcan_results.loc[_gene, _trait] / smultixcan_results[_trait].sum()
)

_trait = "estrogen-receptor negative breast cancer"
_gene = "CFH"
assert (
    _tmp.loc[_gene, _trait]
    == smultixcan_results.loc[_gene, _trait] / smultixcan_results[_trait].sum()
)

_trait = "asthma"
_gene = "C1orf112"
assert (
    _tmp.loc[_gene, _trait]
    == smultixcan_results.loc[_gene, _trait] / smultixcan_results[_trait].sum()
)

In [22]:
smultixcan_results = _tmp

In [23]:
smultixcan_results.head()

,100001_raw-Food_weight,100002_raw-Energy,100003_raw-Protein,100004_raw-Fat,100005_raw-Carbohydrate,100006_raw-Saturated_fat,100007_raw-Polyunsaturated_fat,100008_raw-Total_sugars,100009_raw-Englyst_dietary_fibre,100010-Portion_size,...,visual impairment,vitiligo,vitreous body disease,vocal cord polyp,voice disorders,wellbeing measurement AND family relationship,wheezing,whooping cough,worry measurement,wrist fracture
gene_name,,,,,,,,,,,,,,,,,,,,,
DPM1,0.000060,0.000039,0.000005,0.000016,0.000061,0.000075,0.000004,4.254814e-05,0.000031,0.000017,...,0.000020,0.000075,0.000064,0.000047,0.000066,0.000066,0.000069,0.000058,6.250232e-05,0.000034
SCYL3,0.000033,0.000055,0.000119,0.000041,0.000050,0.000011,0.000059,4.103114e-05,0.000020,0.000036,...,0.000118,0.000007,0.000030,0.000007,0.000029,0.000126,0.000027,0.000114,1.421837e-05,0.000070
C1orf112,0.000027,0.000022,0.000067,0.000023,0.000022,0.000014,0.000067,4.432186e-05,0.000030,0.000041,...,0.000098,0.000055,0.000030,0.000023,0.000084,0.000068,0.000018,0.000024,9.338414e-06,0.000056
FGR,0.000015,0.000014,0.000047,0.000019,0.000003,0.000010,0.000008,5.350929e-07,0.000019,0.000020,...,0.000036,0.000114,0.000152,0.000006,0.000057,0.000038,0.000035,0.000050,1.969101e-05,0.000058
CFH,0.000029,0.000021,0.000039,0.000063,0.000017,0.000018,0.000088,1.668206e-05,0.000017,0.000095,...,0.000014,0.000036,0.000060,0.000038,0.000083,0.000015,0.000005,0.000081,8.895497e-11,0.000036


# Project S-MultiXcan data into MultiPLIER latent space

In [24]:
mproj = MultiplierProjection()

In [25]:
smultixcan_into_multiplier = mproj.transform(smultixcan_results)

In [26]:
smultixcan_into_multiplier.shape

(987, 3752)

In [27]:
smultixcan_into_multiplier.head()

,100001_raw-Food_weight,100002_raw-Energy,100003_raw-Protein,100004_raw-Fat,100005_raw-Carbohydrate,100006_raw-Saturated_fat,100007_raw-Polyunsaturated_fat,100008_raw-Total_sugars,100009_raw-Englyst_dietary_fibre,100010-Portion_size,...,visual impairment,vitiligo,vitreous body disease,vocal cord polyp,voice disorders,wellbeing measurement AND family relationship,wheezing,whooping cough,worry measurement,wrist fracture
LV1,-0.021292,-0.046815,-0.021585,-0.030324,-0.017773,-0.007844,-0.015529,-0.011609,-0.051342,-0.036813,...,-0.001614,0.002102,-0.005318,-0.003041,0.012873,-0.015951,-0.018904,0.009576,0.019755,0.010027
LV2,0.056061,-0.009864,-0.028888,-0.053573,0.006951,-0.054597,-0.030748,0.014519,-0.026933,0.009024,...,-0.029614,-0.033636,0.020768,0.010656,-0.013305,0.050738,0.012085,-0.032372,-0.024540,-0.002616
LV3,0.001650,-0.004262,0.019211,0.007484,0.004548,0.006548,0.006741,-0.005634,-0.022129,-0.017256,...,0.010539,-0.019591,-0.013391,-0.044369,0.020142,0.006052,0.020869,0.043931,0.058049,-0.013144
LV4,0.026265,-0.007235,0.004304,-0.042464,-0.001093,-0.041208,-0.051179,0.009060,-0.003999,-0.005874,...,-0.018162,0.012136,-0.046232,-0.030949,-0.040147,0.044545,0.009865,-0.049828,-0.013725,0.024988
LV5,-0.015616,0.010799,0.017792,0.010591,0.001972,0.040024,-0.027612,-0.001540,0.033072,0.009477,...,0.030632,-0.023253,0.015853,-0.002222,0.019385,-0.015501,-0.031477,-0.002554,-0.032620,0.017731


# Quick analysis

In [28]:
(smultixcan_into_multiplier.loc["LV603"].sort_values(ascending=False).head(20))

30220_raw-Basophill_percentage                                   0.181608
30130_raw-Monocyte_count                                         0.158166
30000_raw-White_blood_cell_leukocyte_count                       0.146061
myeloid white cell count                                         0.133998
leukocyte count                                                  0.131486
sum of neutrophil and eosinophil counts                          0.128405
granulocyte count                                                0.128316
sum of basophil and neutrophil counts                            0.126586
neutrophil count                                                 0.125868
30160-Basophill_count                                            0.122953
30180_raw-Lymphocyte_percentage                                  0.119818
30140_raw-Neutrophill_count                                      0.113055
monocyte count                                                   0.113054
20003_1140875420-Treatmentmedication_c

In [29]:
(smultixcan_into_multiplier.loc["LV136"].sort_values(ascending=False).head(20))

coronary artery disease                                           0.210545
5132_raw-3mm_strong_meridian_right                                0.179597
5134_raw-6mm_strong_meridian_left                                 0.167916
5099_raw-3mm_weak_meridian_right                                  0.161459
5133_raw-6mm_strong_meridian_right                                0.160688
5098_raw-6mm_weak_meridian_right                                  0.157502
I9_CORATHER-Coronary_atherosclerosis                              0.154131
5097_raw-6mm_weak_meridian_left                                   0.150595
5096_raw-3mm_weak_meridian_left                                   0.150400
5135_raw-3mm_strong_meridian_left                                 0.149901
I9_IHD-Ischaemic_heart_disease_wide_definition                    0.148814
I9_MI_STRICT-Myocardial_infarction_strict                         0.125965
I9_MI-Myocardial_infarction                                       0.125965
acute myocardial infarcti

In [30]:
(smultixcan_into_multiplier.loc["LV844"].sort_values(ascending=False).head(20))

celiac disease                                                                                  0.714810
rheumatoid arthritis                                                                            0.600279
K11_COELIAC-Coeliac_disease                                                                     0.600040
malabsorption syndrome                                                                          0.570880
2986-Started_insulin_within_one_year_diagnosis_of_diabetes                                      0.350327
6144_3-Never_eat_eggs_dairy_wheat_sugar_Wheat_products                                          0.338204
systemic lupus erythematosus                                                                    0.327493
hyperthyroidism AND thyrotoxicosis                                                              0.309382
20003_1140883066-Treatmentmedication_code_insulin_product                                       0.289095
psoriasis                                              

# Save

In [31]:
output_file = Path(
    RESULTS_PROJ_OUTPUT_DIR, f"projection-{results_filename_stem}.pkl"
).resolve()

display(output_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/projections/projection-smultixcan-efo_partial-mashr-zscores.pkl')

In [32]:
smultixcan_into_multiplier.to_pickle(output_file)

## RDS format

In [33]:
output_rds_file = output_file.with_suffix(".rds")
display(output_rds_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/projections/projection-smultixcan-efo_partial-mashr-zscores.rds')

In [34]:
with localconverter(ro.default_converter + pandas2ri.converter):
    data_r = ro.conversion.py2rpy(smultixcan_into_multiplier)

In [35]:
data_r

100001_raw-Food_weight,100002_raw-Energy,100003_raw-Protein,...,whooping cough,worry measurement,wrist fracture
-0.021292,-0.046815,-0.021585,...,0.009576,0.019755,0.010027
0.056061,-0.009864,-0.028888,,-0.032372,-0.024540,-0.002616
0.001650,-0.004262,0.019211,,0.043931,0.058049,-0.013144
0.026265,-0.007235,0.004304,,-0.049828,-0.013725,0.024988
...,...,...,,...,...,...
0.015128,-0.001040,-0.006422,,0.025357,-0.021055,-0.014192
-0.018430,0.050533,0.040084,,-0.001305,0.023432,0.015387
0.016253,-0.007979,-0.008175,,-0.005414,0.066239,0.003844
-0.021666,-0.017742,-0.014154,,-0.041389,0.031741,-0.030431


In [36]:
saveRDS(data_r, str(output_rds_file))

<rpy2.rinterface_lib.sexp.NULLType object at 0x7fb5e6448580> [RTYPES.NILSXP]

In [37]:
# testing: load the rds file again
data_r = readRDS(str(output_rds_file))

In [38]:
with localconverter(ro.default_converter + pandas2ri.converter):
    data_again = ro.conversion.rpy2py(data_r)
#     data_again.index = data_again.index.astype(int)

In [39]:
data_again.shape

(987, 3752)

In [40]:
data_again.head()

,100001_raw-Food_weight,100002_raw-Energy,100003_raw-Protein,100004_raw-Fat,100005_raw-Carbohydrate,100006_raw-Saturated_fat,100007_raw-Polyunsaturated_fat,100008_raw-Total_sugars,100009_raw-Englyst_dietary_fibre,100010-Portion_size,...,visual impairment,vitiligo,vitreous body disease,vocal cord polyp,voice disorders,wellbeing measurement AND family relationship,wheezing,whooping cough,worry measurement,wrist fracture
LV1,-0.021292,-0.046815,-0.021585,-0.030324,-0.017773,-0.007844,-0.015529,-0.011609,-0.051342,-0.036813,...,-0.001614,0.002102,-0.005318,-0.003041,0.012873,-0.015951,-0.018904,0.009576,0.019755,0.010027
LV2,0.056061,-0.009864,-0.028888,-0.053573,0.006951,-0.054597,-0.030748,0.014519,-0.026933,0.009024,...,-0.029614,-0.033636,0.020768,0.010656,-0.013305,0.050738,0.012085,-0.032372,-0.024540,-0.002616
LV3,0.001650,-0.004262,0.019211,0.007484,0.004548,0.006548,0.006741,-0.005634,-0.022129,-0.017256,...,0.010539,-0.019591,-0.013391,-0.044369,0.020142,0.006052,0.020869,0.043931,0.058049,-0.013144
LV4,0.026265,-0.007235,0.004304,-0.042464,-0.001093,-0.041208,-0.051179,0.009060,-0.003999,-0.005874,...,-0.018162,0.012136,-0.046232,-0.030949,-0.040147,0.044545,0.009865,-0.049828,-0.013725,0.024988
LV5,-0.015616,0.010799,0.017792,0.010591,0.001972,0.040024,-0.027612,-0.001540,0.033072,0.009477,...,0.030632,-0.023253,0.015853,-0.002222,0.019385,-0.015501,-0.031477,-0.002554,-0.032620,0.017731


In [41]:
pd.testing.assert_frame_equal(
    smultixcan_into_multiplier,
    data_again,
    check_names=False,
    check_exact=True,
    #     rtol=0.0,
    #     atol=1e-50,
    #     check_dtype=False,
)

## Text format

In [42]:
# tsv format
output_text_file = output_file.with_suffix(".tsv.gz")
display(output_text_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/projections/projection-smultixcan-efo_partial-mashr-zscores.tsv.gz')

In [43]:
smultixcan_into_multiplier.to_csv(
    output_text_file, sep="\t", index=True, float_format="%.5e"
)

In [44]:
# testing
# data2 = data.copy()
# data2.index = list(range(0, data2.shape[0]))

data_again = pd.read_csv(output_text_file, sep="\t", index_col=0)

# data_again.index = list(data_again.index)
# data_again["part_k"] = data_again["part_k"].astype(float)

In [45]:
data_again.shape

(987, 3752)

In [46]:
data_again.head()

,100001_raw-Food_weight,100002_raw-Energy,100003_raw-Protein,100004_raw-Fat,100005_raw-Carbohydrate,100006_raw-Saturated_fat,100007_raw-Polyunsaturated_fat,100008_raw-Total_sugars,100009_raw-Englyst_dietary_fibre,100010-Portion_size,...,visual impairment,vitiligo,vitreous body disease,vocal cord polyp,voice disorders,wellbeing measurement AND family relationship,wheezing,whooping cough,worry measurement,wrist fracture
LV1,-0.021292,-0.046815,-0.021585,-0.030324,-0.017773,-0.007844,-0.015529,-0.011609,-0.051342,-0.036813,...,-0.001614,0.002102,-0.005318,-0.003041,0.012873,-0.015951,-0.018904,0.009576,0.019755,0.010027
LV2,0.056061,-0.009864,-0.028888,-0.053573,0.006951,-0.054597,-0.030748,0.014519,-0.026933,0.009024,...,-0.029614,-0.033636,0.020768,0.010656,-0.013305,0.050738,0.012085,-0.032372,-0.024540,-0.002616
LV3,0.001650,-0.004262,0.019211,0.007484,0.004548,0.006548,0.006741,-0.005634,-0.022129,-0.017256,...,0.010539,-0.019591,-0.013391,-0.044369,0.020142,0.006052,0.020869,0.043931,0.058049,-0.013144
LV4,0.026265,-0.007235,0.004304,-0.042464,-0.001093,-0.041208,-0.051179,0.009060,-0.003999,-0.005874,...,-0.018162,0.012136,-0.046232,-0.030949,-0.040147,0.044545,0.009865,-0.049828,-0.013725,0.024988
LV5,-0.015616,0.010799,0.017792,0.010592,0.001972,0.040024,-0.027612,-0.001540,0.033072,0.009477,...,0.030632,-0.023253,0.015853,-0.002222,0.019385,-0.015501,-0.031477,-0.002554,-0.032620,0.017731


In [47]:
pd.testing.assert_frame_equal(
    smultixcan_into_multiplier,
    data_again,
    check_exact=False,
    rtol=0.0,
    atol=5e-5,
)